In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
from datasets import load_dataset

ds = load_dataset("hugsid/legal-contracts")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train_data.csv:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

validation_data.csv:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

test_data.csv:   0%|          | 0.00/6.47M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26539 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3807 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7680 [00:00<?, ? examples/s]

In [3]:
ds["train"].save_to_disk("content/legal-contracts")


Saving the dataset (0/1 shards):   0%|          | 0/26539 [00:00<?, ? examples/s]

In [4]:
from datasets import load_from_disk


dataset = load_from_disk("content/legal-contracts")


print(dataset)
print(dataset[0])


Dataset({
    features: ['text'],
    num_rows: 26539
})
{'text': 'THIS SUPPLEMENT AND JOINDER AGREEMENT (this Agreement), dated as of September 29, 2017, is made among Triangle Capital Corporation, a Maryland corporation (the Borrower), the Guarantors party to the Credit Agreement referred to below, Branch Banking and Trust Company, as administrative agent, swingline lender and an existing Lender pursuant to the Credit Agreement (the Administrative Agent), and United Community Bank, as a new joining Lender pursuant to the Credit Agreement (the Additional Lender).'}


In [5]:
dataset = dataset.filter(lambda x: x["text"] is not None and len(x["text"].strip()) > 50)


Filter:   0%|          | 0/26539 [00:00<?, ? examples/s]

In [6]:
import re

def clean_text(text):

    text = re.sub(r'Page \d+ of \d+', '', text)


    text = text.encode('ascii', 'ignore').decode()

    text = re.sub(r'\n\s*\n', '\n\n', text)


    text = re.sub(r'[ \t]+', ' ', text)


    text = re.sub(r'(?<=\n)([A-Z\s]{3,})(?=\n)', lambda m: m.group(0).title(), text)


    return text.strip()




cleaned_dataset = dataset.map(lambda x: {"text": clean_text(x["text"])})
cleaned_dataset


Map:   0%|          | 0/26539 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 26539
})

In [7]:
from datasets import DatasetDict

split_dataset = cleaned_dataset.train_test_split(test_size=0.1)

In [8]:
def chunk_text(batch, chunk_size=512):
    chunked = []

    for text in batch["text"]:
        words = text.split()
        chunks = [
            {"text": " ".join(words[i:i + chunk_size])}
            for i in range(0, len(words), chunk_size)
        ]
        chunked.extend(chunks)

    return {"text": [c["text"] for c in chunked]}


chunked_dataset = split_dataset["train"].map(
    chunk_text,
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/23885 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"


from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_passages(batch):
    batch["embedding"] = model.encode(batch["text"], show_progress_bar=False).tolist()
    return batch

embedded_dataset = chunked_dataset.map(embed_passages, batched=True)


In [10]:
!pip install hnswlib


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp311-cp311-linux_x86_64.whl size=2389211 sha256=bd7fc253bfb57ec84adcceeabe99a5741530674e01b671ec16a9cb688ffee641
  Stored in directory: /root/.cache/pip/wheels/ea/4e/27/39aebca9958719776e36fada290845a7ef10f053ad70e22ceb
Successfully built hnswlib


In [11]:
import hnswlib
import numpy as np


embeddings = np.array(embedded_dataset["embedding"])
texts = embedded_dataset["text"]


dim = embeddings.shape[1]
num_elements = len(embeddings)

p = hnswlib.Index(space='cosine', dim=dim)
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

p.add_items(embeddings)


p.set_ef(50)


id_to_text = {i: texts[i] for i in range(num_elements)}


def search(query, top_k=5):
    query_embedding = model.encode([query])
    labels, distances = p.knn_query(query_embedding, k=top_k)
    return [(id_to_text[i], 1 - dist) for i, dist in zip(labels[0], distances[0])]


results = search("arbitration or dispute resolution clause", top_k=3)


for idx, (text, score) in enumerate(results):
    print(f"\n🔹 Result {idx+1} (Score: {score:.4f}):\n{text[:500]}...")



🔹 Result 1 (Score: 0.7696):
9\. Resolution of Differences Over Breaches of Agreement. The parties shall use good faith efforts to resolve any controversy or claim arising out of, or relating to this Agreement or the breach thereof. If despite their good faith efforts, the parties are unable to resolve such controversy or claim, then such controversy or claim shall be resolved by arbitration in Chicago, Illinois, with one (1)arbitrator, in accordance with the National Rules for Resolution of Employment Disputes of the Ameri...

🔹 Result 2 (Score: 0.7367):
(a) Agreement to Arbitrate.The Company and Executive hereby agree to resolve by final and binding arbitration any and all claims or controversies in any way arising out of, relating to or associated with Executives employment with the Company or any of its parents, affiliates, or subsidiaries, or the termination of such employment or any breach of this Agreement. This mutual agreement to arbitrate includes any claims that the Company 

In [12]:
!pip install -q langchain groq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00


In [13]:
!pip install langchain-groq


In [14]:
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate


import os
os.environ["GROQ_API_KEY"] = "groq_api_key"


llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")


def format_context(docs):
    return "\n\n".join([f"{i+1}. {doc}" for i, (doc, _) in enumerate(docs)])


def ask_legal_bot(question):
    relevant_docs = search(question, top_k=3)
    context = format_context(relevant_docs)

    prompt = f"""You are a legal assistant. Use the following legal documents to answer the question:

{context}

Q: {question}
A:"""

    messages = [
        SystemMessage(content="You are a helpful legal assistant."),
        HumanMessage(content=prompt)
    ]

    response = llm(messages)
    return response.content

In [15]:
ask_legal_bot("What happens if the borrower defaults?")


<ipython-input-14-438505ccd832>:33: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


'According to the legal documents, if the borrower defaults, it means that the borrower has failed to make a payment when and as due, whether at maturity, by reason of acceleration or otherwise, of interest on any Loan or Reimbursement Obligation or the payment of any other Obligation, and such default has continued for a period of three Business Days.'

In [16]:
ask_legal_bot("Can either party terminate the contract early?")

'Yes, either party can terminate the contract early under certain circumstances.\n\nAccording to Section 3.4(b), the contract can be terminated if the Closing does not occur on or before September 8, 2016 (or a later date agreed upon in writing by the parties). However, this right to terminate is not available to a party whose breach of the contract has resulted in the failure of the Closing to occur.\n\nAdditionally, Section 6.1 states that the contract can be terminated by written notice if the Closing has not been consummated by the third Business Day following the date of the agreement.\n\nFinally, Section 9 provides that the contract can be terminated if an event of default occurs and remains uncured after the curative period has expired.\n\nIn all cases, termination of the contract does not relieve either party of its liabilities accrued up to the time of termination, and does not prejudice the right of either party to recover all monies then due, to enforce the contract, and to 

In [17]:
ask_legal_bot("What penalties are there for breach of contract?")

"Based on the provided legal documents, the penalties for breach of contract are as follows:\n\n1. The Company may apply for and receive an injunction without bond to restrain any violation of the Separation Agreement (Section 11(i)).\n2. The Company may cease providing termination payments or benefits under the Separation Agreement (Section 11(ii)).\n3. The Employee may be obligated to pay the Company's costs and expenses in enforcing its rights (Section 11(iii)).\n4. The Company may recover and/or cease providing consideration to the Employee under the Agreement (Section 15).\n5. The Company may obtain damages, except as provided by law (Section 15).\n\nAdditionally, the Company may also seek to recover its attorneys' fees in enforcing its rights under the Agreement (Section 15)."